# Word-Level Tokenizing

If not present, download the dataset = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt" to `data/01_raw/shakespeare.txt`

In [12]:
%load_ext kedro.ipython
%reload_kedro

from typing import Any, Dict, List, Tuple

import re

from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[02/21/24 13:04:43] INFO     Resolved project path as: /Users/b/Documents/ML/tolkenizers.           ]8;id=102699;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=467699;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#149\149]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/21/24 13:04:44] INFO     Kedro project tolkenizers                                              ]8;id=599292;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=796079;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#119\119]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=361259;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=392065;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#120\120]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=748231;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=45894;file:///Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/kedro/ipython/__init__.py#126\126]8;;\

# Data

In [2]:
shakespeare = load_dataset("tiny_shakespeare")
train_split = shakespeare["train"]
test_split = shakespeare["test"]

# split on words and specific punctuation

In [3]:


s = shakespeare['train']['text'][0][:100]

In [4]:
s

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [5]:
delimiters = r";|,|\n|'|`| "
pat = re.compile(delimiters)
t = re.split(pat, s[:100])
print(t)

['First', 'Citizen:', 'Before', 'we', 'proceed', 'any', 'further', '', 'hear', 'me', 'speak.', '', 'All:', 'Speak', '', 'speak.', '', 'First', 'Citizen:', 'You']


Though we don't have any contractions, we can see words are roughly whole. The next step, would be to assign a unique integer to each word. 
For us, a good-enough is the entire dataset vocabulary.

In [6]:
count_vec = CountVectorizer(analyzer='word', tokenizer=lambda x: re.split(pat, x), lowercase=False, stop_words=None)
count_vec.fit(shakespeare['train']['text'])
# print the vocabulary encoding values, sorted
print(f"{len(count_vec.vocabulary_) = }")
{k: v for k, v in sorted(count_vec.vocabulary_.items(), key=lambda item: item[1])}

[02/21/24 13:04:13] WARNING  /Users/b/.pyenv/versions/3.11.7/envs/tok/lib/python3.11/site-packages/ ]8;id=602527;file:///Users/b/.pyenv/versions/3.11.7/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=701157;file:///Users/b/.pyenv/versions/3.11.7/lib/python3.11/warnings.py#109\109]8;;\
                             sklearn/feature_extraction/text.py:525: UserWarning: The parameter                    
                             'token_pattern' will not be used since 'tokenizer' is not None'                       
                               warnings.warn(                                                                      
                                                                                                                   

len(count_vec.vocabulary_) = 18120



{
    '': 0,
    '!': 1,
    '&C:': 2,
    '&c.': 3,
    '--': 4,
    '--Hold': 5,
    '--I': 6,
    '--O': 7,
    '--Plague': 8,
    '--Tybalt': 9,
    '--Where': 10,
    '--a': 11,
    '--an': 12,
    '--and': 13,
    '--as': 14,
    '--be': 15,
    '--believe': 16,
    '--but': 17,
    '--cast': 18,
    '--cousin': 19,
    '--do': 20,
    '--for': 21,
    '--give': 22,
    '--goddess!--O': 23,
    '--here': 24,
    '--how': 25,
    '--if': 26,
    '--it': 27,
    '--let': 28,
    '--like': 29,
    '--mightst': 30,
    '--or': 31,
    '--part': 32,
    '--poor': 33,
    '--so': 34,
    '--that': 35,
    '--the': 36,
    '--there': 37,
    '--thine': 38,
    '--this': 39,
    '--thou': 40,
    '--thus': 41,
    '--we': 42,
    '--what': 43,
    '--which': 44,
    '--whither': 45,
    '--whom': 46,
    '--will': 47,
    '--you': 48,
    '--your': 49,
    '3': 50,
    ':': 51,
    '?': 52,
    'A': 53,
    'ABHORSON:': 54,
    'ABRAHAM:': 55,
    'AEdile:': 56,
    'AEdiles': 57,
    '

In [7]:
count_vec.vocabulary_['plague']

12682

In [8]:
def encoding(s: List[str], cv) -> List[int]:
    """s is the pre-split string. """
    vocab = cv.vocabulary_
    encoded = [vocab[tok] for tok in s if tok in vocab]
    return encoded

In [9]:
print(t)
my_encoding = encoding(t, count_vec)
print(my_encoding)

['First', 'Citizen:', 'Before', 'we', 'proceed', 'any', 'further', '', 'hear', 'me', 'speak.', '', 'All:', 'Speak', '', 'speak.', '', 'First', 'Citizen:', 'You']
[1051, 592, 327, 17454, 13052, 3702, 8521, 0, 9236, 11141, 15103, 0, 155, 2595, 0, 15103, 0, 1051, 592, 3175]


Note the `0`s, those are the spaces. 

## Thoughts

The idea is interesting, each word hold a _lot_ of contextual information. That is, each word, even a compound word, feel "unique" to the space. In the case of the Shakespeare dataset, this is `N=19467` unique tokens given our delimiter splitting. A real English unigram vocabulary could be hundreds of thousands of tokens in cardinality. 

Limits: lack of shared meaning across similar words. `count_vec.vocabulary_['plagues']=13672` while `plague=13669`. (numeric distance does not represent similarity)

### Stopwords

We may want to keep vocabulary smaller. We can drop "stop" words. words that connect or are frequently used. Stop words are words like “and”, “the”, “him”, which are presumed to be uninformative in representing the content of a text, and which may be removed to avoid them being construed as signal for prediction.

In [13]:
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /Users/b/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
